In [36]:
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,SimpleRNN,BatchNormalization,GRU
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
import pickle
import os




In [148]:
class StockPredictor:
    def __init__(self,stock_name='AAPL',interval="1h",period="2y",split_ratio=0.9,window_size=6):
        self.stock_name = stock_name
        self.split_ratio = split_ratio
        self.period = period
        self.interval = interval
        self.window_size = window_size
        self.data = yf.download(self.stock_name, period=self.period,interval=self.interval)
        self.data = self.data.dropna()
        cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        self.data.columns = cols
    
    def get_stock_data(self):
        stock_data = yf.download(self.stock_name, period=self.period,interval=self.interval)
        stock_data = stock_data.dropna()
        cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        stock_data.columns = cols
        return stock_data
    
    def window_data_univariate(self,data, window_size):
        X = []
        y = []
        
        for i in range(len(data) - window_size - 1):
            X.append(data[i:(i + window_size)])
            y.append(data[i + window_size])
        print(np.array(X).shape)
        return np.array(X), np.array(y).reshape(-1, 1)
    
    def window_data_multivariate(self,data,close_data, window_size):
        X = []
        y = []
        
        for i in range(len(data) - window_size - 1):
            # Window includes all features
            X.append(data[i:(i + window_size), :])
            # Target is next day's closing price
            y.append(close_data[i + window_size])  # Closing price is first column
            
        return np.array(X), np.array(y).reshape(-1, 1)
    
    def build_lstm_model(self,X_train):
        model = Sequential([
            LSTM(units=128, 
            return_sequences=True, 
            input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        
        LSTM(units=64),
        Dropout(0.2),
        
        Dense(32, activation='relu'),
        Dense(1)
        ])
        optimizer = Adam(learning_rate=0.01)
        model.compile(optimizer=optimizer, loss='mean_squared_error')
        return model
    
    def build_mlp_model(self):
        model_mlp = MLPRegressor(hidden_layer_sizes=(128, 64), max_iter=100, verbose=True)
        return model_mlp
    
    def get_current_price(self):
        stock_data = yf.download(self.stock_name, period='2y',interval='1h')
        return stock_data['Close'].to_numpy()[-1]
    
    def train_lstm_univariate(self):
        scaler = MinMaxScaler()
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        print(df_windowed.shape,df_target.shape,df_windowed_reshaped.shape)
        x_scaler = scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        train_size = int(self.split_ratio*len(df_windowed))
        X_train = df_windowed[:train_size]
        y_train = df_target[:train_size]
        X_test = df_windowed[train_size:]
        y_test = df_target[train_size:]
        model = self.build_lstm_model(X_train)
        model.fit(X_train, y_train, epochs=100,batch_size=32,    
            validation_split=0.1,
            verbose=1)

        model.save(f'models/{self.stock_name}/LSTM_univariate.h5')
     
    def train_lstm_multivariante(self):
        scaler = MinMaxScaler()
        data = self.data[['Close', 'High', 'Low', 'Volume']]
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_multivariate(data.to_numpy(),data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        train_size = int(self.split_ratio*len(df_windowed))
        X_train = df_windowed[:train_size]
        y_train = df_target[:train_size]
        X_test = df_windowed[train_size:]
        y_test = df_target[train_size:]
        model = self.build_lstm_model(X_train)
        model.fit(X_train, y_train, epochs=100,batch_size=32,    
            validation_split=0.1,
            verbose=1)

        model.save(f'models/{self.stock_name}/LSTM_multivariate.h5')   
        
    def train_mlp_univariante(self):
        scaler = MinMaxScaler()
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        print(df_windowed.shape,df_target.shape,df_windowed_reshaped.shape)
        x_scaler = scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        train_size = int(self.split_ratio*len(df_windowed))
        X_train = df_windowed[:train_size]
        y_train = df_target[:train_size]
        X_test = df_windowed[train_size:]
        y_test = df_target[train_size:]
        model = self.build_mlp_model()
        model.fit(X_train.reshape(X_train.shape[0],-1), y_train)
        output_dir = f'models/{self.stock_name}'
        os.makedirs(output_dir, exist_ok=True)
        with open(f'models/{self.stock_name}/MLP_univaraite.pkl', 'wb') as file:
            pickle.dump(model, file)
    
    
    def train(self):
        self.train_lstm_univariate()
        self.train_lstm_multivariante()
        self.train_mlp_univariante()
    
    def forecast_lstm_univariante(self,n_hours:int=7):
        model_dir = f'models/{self.stock_name}/LSTM_univariate.h5'
        model = tf.keras.models.load_model(model_dir)
        ## forecast for n hours
        print(model_dir)
        forecast = []
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        for i in range(n_hours):
            X = X.reshape(1, X.shape[0], X.shape[1])
            y_pred = model.predict(X)
            forecast.append(y_pred)
            print(forecast)
            X = np.concatenate((X[0][1:], y_pred))
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    def forecast_lstm_multivariante(self,n_hours:int=7):
        model_dir = f'models/{self.stock_name}/LSTM_multivariate.h5'
        model = tf.keras.models.load_model(model_dir)
        ## forecast for n hours
        print(model_dir)
        forecast = []
        data = self.data[['Close', 'High', 'Low', 'Volume']]
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_multivariate(data.to_numpy(),data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        for i in range(n_hours):
            X = X.reshape(1, X.shape[0], X.shape[1])
            y_pred = model.predict(X)
            forecast.append(y_pred)
            y_pred_expanded = X[-1].copy()  # Use the last timestep as a template
            y_pred_expanded[0] = y_pred[0, 0]  # Update the 'Close' feature with the prediction
            X = np.concatenate((X[0][1:], y_pred_expanded), axis=0)
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    def forecast_mlp_univariate(self,n_hours):
        model_dir = f'models/{self.stock_name}/MLP_univaraite.pkl'
        with open(model_dir, 'rb') as file:
            model = pickle.load(file)
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        forecast = []
        for i in range(n_hours):
            y_pred = model.predict(X.reshape(1, -1)).reshape(-1, 1)
            forecast.append(y_pred)
            X = np.concatenate((X[1:], y_pred))
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    
    def forecast_nhours(self,n_hours:int=7):
        return {
            'LSTM_univariate':self.forecast_lstm_univariante(n_hours),
            'LSTM_multivariate':self.forecast_lstm_multivariante(n_hours),
            'MLP_univariate':self.forecast_mlp_univariate(n_hours)
        }
    

        
        
            

In [149]:
predictor = StockPredictor("AAPL")

[*********************100%***********************]  1 of 1 completed


In [150]:
predictor.forecast_mlp_univariate(7)

(3494, 6, 1)


array([[242.55432281],
       [242.3967911 ],
       [242.5014088 ],
       [242.58331609],
       [242.56318562],
       [242.62003063],
       [242.621854  ]])

In [151]:
predictor.train()

(3494, 6, 1)
(3494, 6, 1) (3494, 1) (3494, 6)
Epoch 1/100


c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0570 - val_loss: 4.0247e-04
Epoch 2/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.4623e-04 - val_loss: 7.2613e-04
Epoch 3/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.0085e-04 - val_loss: 9.7912e-04
Epoch 4/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 5.2982e-04 - val_loss: 2.5444e-04
Epoch 5/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.6219e-04 - val_loss: 0.0010
Epoch 6/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.6407e-04 - val_loss: 4.0176e-04
Epoch 7/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.6025e-04 - val_loss: 2.8238e-04
Epoch 8/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.5807e-04 - val_loss: 2.3727e-04
Epoch 9/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.8439e-04 - val_loss: 1.9020e-04
Epoch 10/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.1413e-04 - val_loss: 1.8869e-04
Epoch 11/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.9148e-04 - val_loss: 2.083

Epoch 1/100


c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0267 - val_loss: 7.2568e-04
Epoch 2/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 8.3404e-04 - val_loss: 1.9855e-04
Epoch 3/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.0331e-04 - val_loss: 2.3784e-04
Epoch 4/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.5160e-04 - val_loss: 2.4048e-04
Epoch 5/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.1755e-04 - val_loss: 1.8123e-04
Epoch 6/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.7053e-04 - val_loss: 6.3580e-04
Epoch 7/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.4340e-04 - val_loss: 7.7577e-04
Epoch 8/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.4647e-04 - val_loss: 1.7635e-04
Epoch 9/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.5584e-04 - val_loss: 4.7328e-04
Epoch 10/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 4.8545e-04 - val_loss: 0.0016
Epoch 11/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.5840e-04 - val_loss: 3.

(3494, 6, 1)
(3494, 6, 1) (3494, 1) (3494, 6)
Iteration 1, loss = 0.00958456


c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 2, loss = 0.00069286
Iteration 3, loss = 0.00023975
Iteration 4, loss = 0.00013172
Iteration 5, loss = 0.00011230
Iteration 6, loss = 0.00010169
Iteration 7, loss = 0.00010072
Iteration 8, loss = 0.00009753
Iteration 9, loss = 0.00009350
Iteration 10, loss = 0.00009189
Iteration 11, loss = 0.00009030
Iteration 12, loss = 0.00008843
Iteration 13, loss = 0.00008727
Iteration 14, loss = 0.00008497
Iteration 15, loss = 0.00008286
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [153]:
predictor.forecast_nhours(7)

models/AAPL/LSTM_univariate.h5
(3494, 6, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
[array([[0.7033186]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[array([[0.7033186]], dtype=float32), array([[0.70260465]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[array([[0.7033186]], dtype=float32), array([[0.70260465]], dtype=float32), array([[0.7017855]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[array([[0.7033186]], dtype=float32), array([[0.70260465]], dtype=float32), array([[0.7017855]], dtype=float32), array([[0.69927305]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.7033186]], dtype=float32), array([[0.70260465]], dtype=float32), array([[0.7017855]], dtype=float32), array([[0.69927305]], dtype=float32), array([[0.6918009]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.7033186]], dtype=float32), array([[0.70260465]], dtype=float32), array([[0.7017855]], dtype=float32), array([[0.69927305]], dtype=float32), array([[0.

[array([[0.7033186]], dtype=float32), array([[0.70260465]], dtype=float32), array([[0.7017855]], dtype=float32), array([[0.69927305]], dtype=float32), array([[0.6918009]], dtype=float32), array([[0.6781872]], dtype=float32), array([[0.6607209]], dtype=float32)]
models/AAPL/LSTM_multivariate.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
(3494, 6, 1)


{'LSTM_univariate': array([[219.18053],
        [219.08467],
        [218.97472],
        [218.63742],
        [217.63428],
        [215.80664],
        [213.46179]], dtype=float32),
 'LSTM_multivariate': array([[240.07625],
        [238.7717 ],
        [238.35303],
        [238.28319],
        [238.26955],
        [238.26898],
        [238.26898]], dtype=float32),
 'MLP_univariate': array([[240.10460801],
        [240.01756803],
        [239.43131158],
        [239.259672  ],
        [238.52353024],
        [237.83666442],
        [237.43627278]])}

In [154]:
class Predictor:
    def __init__(self,interval="1h",period="2y",stocks=['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA']):
        self.stocks = stocks
        self.predictors = {}
        for stock in stocks:
            self.predictors[stock] = StockPredictor(stock_name=stock,interval=interval,period=period)
        
    def train(self):
        for stock in self.predictors.values():
            stock.train()
    
    def forecast_nhours(self,n_hours:int=7):
        forecast = {}
        for stock in self.predictors.values():
            forecast[stock.stock_name] = stock.forecast_nhours(n_hours)
        return forecast
    
    def get_current_prices(self):
        prices = {}
        for stock in self.predictors.values():
            prices[stock.stock_name] = stock.get_current_price()
        return prices
    

In [ ]:
pred = Predictor()
pred.train()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

(3494, 6, 1)
(3494, 6, 1) (3494, 1) (3494, 6)
Epoch 1/100



c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0539 - val_loss: 7.8669e-04
Epoch 2/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 7.8024e-04 - val_loss: 4.5837e-04
Epoch 3/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.4178e-04 - val_loss: 2.8173e-04
Epoch 4/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.5062e-04 - val_loss: 4.1672e-04
Epoch 5/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.3857e-04 - val_loss: 2.3194e-04
Epoch 6/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.1165e-04 - val_loss: 3.2085e-04
Epoch 7/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.7029e-04 - val_loss: 3.3125e-04
Epoch 8/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.2133e-04 - val_loss: 6.6544e-04
Epoch 9/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.0831e-04 - val_loss: 0.0011
Epoch 10/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.5766e-04 - val_loss: 4.3256e-04
Epoch 11/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.4972e-04 - val_loss: 3.006

Epoch 1/100


...

c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0083 - val_loss: 3.7462e-04
Epoch 2/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.7126e-04 - val_loss: 7.8519e-04
Epoch 3/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.7498e-04 - val_loss: 9.9861e-05
Epoch 4/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.5504e-04 - val_loss: 8.8418e-05
Epoch 5/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2.0188e-04 - val_loss: 5.8574e-04
Epoch 6/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.9456e-04 - val_loss: 1.0014e-04
Epoch 7/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.8706e-04 - val_loss: 2.7003e-04
Epoch 8/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.5508e-04 - val_loss: 6.7269e-05
Epoch 9/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.6738e-04 - val_loss: 9.9007e-05
Epoch 10/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.7121e-04 - val_loss: 9.6665e-05
Epoch 11/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.1601e-04 - val_loss: 1

(3494, 6, 1)
(3494, 6, 1) (3494, 1) (3494, 6)
Iteration 1, loss = 0.00114229
Iteration 2, loss = 0.00020692
Iteration 3, loss = 0.00011201


c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 4, loss = 0.00009536
Iteration 5, loss = 0.00009203
Iteration 6, loss = 0.00008802
Iteration 7, loss = 0.00007818
Iteration 8, loss = 0.00007261
Iteration 9, loss = 0.00006963
Iteration 10, loss = 0.00006617
Iteration 11, loss = 0.00006418
Iteration 12, loss = 0.00006532
Iteration 13, loss = 0.00005910
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [156]:
pred.forecast_nhours(7)

models/AAPL/LSTM_univariate.h5
(3494, 6, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
[array([[0.8398307]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[array([[0.8398307]], dtype=float32), array([[0.8199195]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[array([[0.8398307]], dtype=float32), array([[0.8199195]], dtype=float32), array([[0.80907553]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.8398307]], dtype=float32), array([[0.8199195]], dtype=float32), array([[0.80907553]], dtype=float32), array([[0.7973914]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[array([[0.8398307]], dtype=float32), array([[0.8199195]], dtype=float32), array([[0.80907553]], dtype=float32), array([[0.7973914]], dtype=float32), array([[0.78437304]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[array([[0.8398307]], dtype=float32), array([[0.8199195]], dtype=float32), array([[0.80907553]], dtype=float32), array([[0.7973914]], dtype=float32), array([[0.784

[array([[0.8398307]], dtype=float32), array([[0.8199195]], dtype=float32), array([[0.80907553]], dtype=float32), array([[0.7973914]], dtype=float32), array([[0.78437304]], dtype=float32), array([[0.77340627]], dtype=float32), array([[0.7605931]], dtype=float32)]
models/AAPL/LSTM_multivariate.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
(3494, 6, 1)


models/AMZN/LSTM_univariate.h5
(3494, 6, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
[array([[0.8781831]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[array([[0.8781831]], dtype=float32), array([[0.8607826]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[array([[0.8781831]], dtype=float32), array([[0.8607826]], dtype=float32), array([[0.8458765]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[array([[0.8781831]], dtype=float32), array([[0.8607826]], dtype=float32), array([[0.8458765]], dtype=float32), array([[0.82993287]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
[array([[0.8781831]], dtype=float32), array([[0.8607826]], dtype=float32), array([[0.8458765]], dtype=float32), array([[0.82993287]], dtype=float32), array([[0.815562]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[array([[0.8781831]], dtype=float32), array([[0.8607826]], dtype=float32), array([[0.8458765]], dtype=float32), array([[0.82993287]], dtype=float32), array([[0.815562

models/AMZN/LSTM_multivariate.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


(3494, 6, 1)
models/GOOGL/LSTM_univariate.h5
(3494, 6, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
[array([[0.9679951]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[array([[0.9679951]], dtype=float32), array([[0.9804182]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.9679951]], dtype=float32), array([[0.9804182]], dtype=float32), array([[0.990829]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[array([[0.9679951]], dtype=float32), array([[0.9804182]], dtype=float32), array([[0.990829]], dtype=float32), array([[1.0009707]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[array([[0.9679951]], dtype=float32), array([[0.9804182]], dtype=float32), array([[0.990829]], dtype=float32), array([[1.0009707]], dtype=float32), array([[1.0108943]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[array([[0.9679951]], dtype=float32), array([[0.9804182]], dtype=float32), array([[0.990829]], dtype=float32), array([[1.0009707]], dtype=float32), array([[

...

(3494, 6, 1)
models/MSFT/LSTM_univariate.h5
(3494, 6, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
[array([[0.8307117]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.8307117]], dtype=float32), array([[0.84035534]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[array([[0.8307117]], dtype=float32), array([[0.84035534]], dtype=float32), array([[0.8496222]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[array([[0.8307117]], dtype=float32), array([[0.84035534]], dtype=float32), array([[0.8496222]], dtype=float32), array([[0.85939205]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.8307117]], dtype=float32), array([[0.84035534]], dtype=float32), array([[0.8496222]], dtype=float32), array([[0.85939205]], dtype=float32), array([[0.869464]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[array([[0.8307117]], dtype=float32), array([[0.84035534]], dtype=float32), array([[0.8496222]], dtype=float32), array([[0.85939205]], dtype=float32)

models/MSFT/LSTM_multivariate.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
(3494, 6, 1)


models/TSLA/LSTM_univariate.h5
(3494, 6, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
[array([[0.6418495]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.6418495]], dtype=float32), array([[0.5704757]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[array([[0.6418495]], dtype=float32), array([[0.5704757]], dtype=float32), array([[0.5455927]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[array([[0.6418495]], dtype=float32), array([[0.5704757]], dtype=float32), array([[0.5455927]], dtype=float32), array([[0.5178375]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[array([[0.6418495]], dtype=float32), array([[0.5704757]], dtype=float32), array([[0.5455927]], dtype=float32), array([[0.5178375]], dtype=float32), array([[0.49103987]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.6418495]], dtype=float32), array([[0.5704757]], dtype=float32), array([[0.5455927]], dtype=float32), array([[0.5178375]], dtype=float32), array([[0.4910398

models/TSLA/LSTM_multivariate.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
(3494, 6, 1)


{'AAPL': {'LSTM_univariate': array([[237.50728],
         [234.83421],
         [233.3784 ],
         [231.8098 ],
         [230.06209],
         [228.5898 ],
         [226.86964]], dtype=float32),
  'LSTM_multivariate': array([[242.07079],
         [241.21873],
         [240.95132],
         [240.92964],
         [240.92944],
         [240.92944],
         [240.92944]], dtype=float32),
  'MLP_univariate': array([[241.90006261],
         [241.7587063 ],
         [241.83934935],
         [241.66985386],
         [241.51264937],
         [241.34471814],
         [241.20153674]])},
 'AMZN': {'LSTM_univariate': array([[213.83368],
         [211.2102 ],
         [208.9628 ],
         [206.55898],
         [204.39229],
         [202.51495],
         [200.98424]], dtype=float32),
  'LSTM_multivariate': array([[201.93044],
         [200.53749],
         [200.22893],
         [200.22719],
         [200.22723],
         [200.22723],
         [200.22723]], dtype=float32),
  'MLP_univariate': arra

In [157]:
pred.get_current_prices()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'AAPL': array([243.30000305]),
 'AMZN': array([224.19999695]),
 'GOOGL': array([191.77999878]),
 'MSFT': array([423.3500061]),
 'TSLA': array([410.76998901])}

In [4]:
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,SimpleRNN,BatchNormalization,GRU
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
import pickle
import os


class StockPredictor:
    def __init__(self,stock_name='AAPL',interval="1h",period="2y",split_ratio=0.9,window_size=6):
        self.stock_name = stock_name
        self.split_ratio = split_ratio
        self.period = period
        self.interval = interval
        self.window_size = window_size
        self.data = yf.download(self.stock_name, period=self.period,interval=self.interval)
        self.data = self.data.dropna()
        cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        self.data.columns = cols
    
    def get_stock_data(self):
        stock_data = yf.download(self.stock_name, period=self.period,interval=self.interval)
        stock_data = stock_data.dropna()
        cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        stock_data.columns = cols
        return stock_data
    
    def window_data_univariate(self,data, window_size):
        X = []
        y = []
        
        for i in range(len(data) - window_size - 1):
            X.append(data[i:(i + window_size)])
            y.append(data[i + window_size])
        print(np.array(X).shape)
        return np.array(X), np.array(y).reshape(-1, 1)
    
    def window_data_multivariate(self,data,close_data, window_size):
        X = []
        y = []
        
        for i in range(len(data) - window_size - 1):
            # Window includes all features
            X.append(data[i:(i + window_size), :])
            # Target is next day's closing price
            y.append(close_data[i + window_size])  # Closing price is first column
            
        return np.array(X), np.array(y).reshape(-1, 1)
    
    def build_lstm_model(self,X_train):
        model = Sequential([
            LSTM(units=128, 
            return_sequences=True, 
            input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        
        LSTM(units=64),
        Dropout(0.2),
        
        Dense(32, activation='relu'),
        Dense(1)
        ])
        optimizer = Adam(learning_rate=0.01)
        model.compile(optimizer=optimizer, loss='mean_squared_error')
        return model
    
    def build_mlp_model(self):
        model_mlp = MLPRegressor(hidden_layer_sizes=(128, 64), max_iter=100, verbose=True)
        return model_mlp
    
    def get_current_price(self):
        stock_data = yf.download(self.stock_name, period='2y',interval='1h')
        return stock_data['Close'].to_numpy()[-1]
    
    def preprocess_data(self,df_windowed,df_target):
        scaler = MinMaxScaler()
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        print(df_windowed.shape,df_target.shape,df_windowed_reshaped.shape)
        x_scaler = scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        train_size = int(self.split_ratio*len(df_windowed))
        X_train = df_windowed[:train_size]
        y_train = df_target[:train_size]
        X_test = df_windowed[train_size:]
        y_test = df_target[train_size:]
        return X_train,y_train,X_test,y_test
        
    def train_lstm_univariate(self):
        scaler = MinMaxScaler()
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        X_train,y_train,X_test,y_test = self.preprocess_data(df_windowed,df_target)
        model = self.build_lstm_model(X_train)
        model.fit(X_train, y_train, epochs=100,batch_size=32,    
            validation_split=0.1,
            verbose=1)

        model.save(f'models/{self.stock_name}/LSTM_univariate.h5')
    

    
    def train_lstm_multivariante(self):
        scaler = MinMaxScaler()
        data = self.data[['Close', 'High', 'Low', 'Volume']]
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_multivariate(data.to_numpy(),data_close.to_numpy().reshape(-1,1),self.window_size)
        X_train,y_train,X_test,y_test = self.preprocess_data(df_windowed,df_target)
        model = self.build_lstm_model(X_train)
        model.fit(X_train, y_train, epochs=100,batch_size=32,    
            validation_split=0.1,
            verbose=1)

        model.save(f'models/{self.stock_name}/LSTM_multivariate.h5')   
        
    def train_mlp_univariante(self):
        scaler = MinMaxScaler()
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        X_train,y_train,X_test,y_test = self.preprocess_data(df_windowed,df_target)
        model = self.build_mlp_model()
        model.fit(X_train.reshape(X_train.shape[0],-1), y_train)
        output_dir = f'models/{self.stock_name}'
        os.makedirs(output_dir, exist_ok=True)
        with open(f'models/{self.stock_name}/MLP_univaraite.pkl', 'wb') as file:
            pickle.dump(model, file)
    
    
    def train(self):
        self.train_lstm_univariate()
        self.train_lstm_multivariante()
        self.train_mlp_univariante()
    
    def forecast_lstm_univariante(self,n_hours:int=7):
        model_dir = f'models/{self.stock_name}/LSTM_univariate.h5'
        model = tf.keras.models.load_model(model_dir)
        ## forecast for n hours
        print(model_dir)
        forecast = []
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        for i in range(n_hours):
            X = X.reshape(1, X.shape[0], X.shape[1])
            y_pred = model.predict(X)
            forecast.append(y_pred)
            print(forecast)
            X = np.concatenate((X[0][1:], y_pred))
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    def forecast_lstm_multivariante(self,n_hours:int=7):
        model_dir = f'models/{self.stock_name}/LSTM_multivariate.h5'
        model = tf.keras.models.load_model(model_dir)
        ## forecast for n hours
        print(model_dir)
        forecast = []
        data = self.data[['Close', 'High', 'Low', 'Volume']]
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_multivariate(data.to_numpy(),data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        for i in range(n_hours):
            X = X.reshape(1, X.shape[0], X.shape[1])
            y_pred = model.predict(X)
            forecast.append(y_pred)
            y_pred_expanded = X[-1].copy()  # Use the last timestep as a template
            y_pred_expanded[0] = y_pred[0, 0]  # Update the 'Close' feature with the prediction
            X = np.concatenate((X[0][1:], y_pred_expanded), axis=0)
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    def forecast_mlp_univariate(self,n_hours):
        model_dir = f'models/{self.stock_name}/MLP_univaraite.pkl'
        with open(model_dir, 'rb') as file:
            model = pickle.load(file)
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        forecast = []
        for i in range(n_hours):
            y_pred = model.predict(X.reshape(1, -1)).reshape(-1, 1)
            forecast.append(y_pred)
            X = np.concatenate((X[1:], y_pred))
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    
    def forecast_nhours(self,n_hours:int=7):
        return {
            'LSTM_univariate':self.forecast_lstm_univariante(n_hours),
            'LSTM_multivariate':self.forecast_lstm_multivariante(n_hours),
            'MLP_univariate':self.forecast_mlp_univariate(n_hours)
        }
    

        
        
            
            
class Predictor:
    def __init__(self,interval="1h",period="2y",stocks=['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA']):
        self.stocks = stocks
        self.predictors = {}
        for stock in stocks:
            self.predictors[stock] = StockPredictor(stock_name=stock,interval=interval,period=period)
        
    def train(self):
        for stock in self.predictors.values():
            stock.train()
    
    def forecast_nhours(self,n_hours:int=7):
        forecast = {}
        for stock in self.predictors.values():
            forecast[stock.stock_name] = stock.forecast_nhours(n_hours)
        return forecast
    
    def get_current_prices(self):
        prices = {}
        for stock in self.predictors.values():
            prices[stock.stock_name] = stock.get_current_price()
        return prices
    

In [8]:
test = Predictor()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [9]:
test.get_current_prices()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'AAPL': array([243.30000305]),
 'AMZN': array([224.19999695]),
 'GOOGL': array([191.77999878]),
 'MSFT': array([423.3500061]),
 'TSLA': array([410.76998901])}

In [7]:
test.forecast_nhours(7)

models/AAPL/LSTM_univariate.h5
(3494, 6, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
[array([[0.85015935]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[array([[0.85015935]], dtype=float32), array([[0.8293338]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[array([[0.85015935]], dtype=float32), array([[0.8293338]], dtype=float32), array([[0.8150362]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[array([[0.85015935]], dtype=float32), array([[0.8293338]], dtype=float32), array([[0.8150362]], dtype=float32), array([[0.8019947]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[array([[0.85015935]], dtype=float32), array([[0.8293338]], dtype=float32), array([[0.8150362]], dtype=float32), array([[0.8019947]], dtype=float32), array([[0.78742707]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[array([[0.85015935]], dtype=float32), array([[0.8293338]], dtype=float32), array([[0.8150362]], dtype=float32), array([[0.8019947]], dtype=float32), array([[0.7

models/AAPL/LSTM_multivariate.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
(3494, 6, 1)


{'LSTM_univariate': array([[238.8939 ],
        [236.09807],
        [234.1786 ],
        [232.4278 ],
        [230.47209],
        [228.77925],
        [226.90611]], dtype=float32),
 'LSTM_multivariate': array([[242.43036],
        [240.85394],
        [240.40614],
        [240.37178],
        [240.37083],
        [240.37083],
        [240.37083]], dtype=float32),
 'MLP_univariate': array([[239.92619   ],
        [239.5743315 ],
        [239.06332438],
        [238.84948458],
        [238.55428402],
        [238.00527909],
        [237.31271738]])}

In [2]:
import yfinance as yf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,SimpleRNN,BatchNormalization,GRU
from tensorflow.keras.optimizers import Adam
import datetime as dt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
import pickle
import os
import json


class StockPredictor:
    def __init__(self,stock_name='AAPL',interval="1h",period="2y",split_ratio=1,window_size=6):
        self.stock_name = stock_name
        self.split_ratio = split_ratio
        self.period = period
        self.interval = interval
        self.window_size = window_size
        self.data = yf.download(self.stock_name, period=self.period,interval=self.interval)
        self.data = self.data.dropna()
        self.data.columns = self.data.columns.droplevel(1) 
    
    def get_stock_data(self):
        stock_data = yf.download(self.stock_name, period=self.period,interval=self.interval)
        stock_data = stock_data.dropna()
        cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        stock_data.columns = cols
        return stock_data
    
    def window_data_univariate(self,data, window_size):
        X = []
        y = []
        
        for i in range(len(data) - window_size - 1):
            X.append(data[i:(i + window_size)])
            y.append(data[i + window_size])
        print(np.array(X).shape)
        return np.array(X), np.array(y).reshape(-1, 1)
    
    def window_data_multivariate(self,data,close_data, window_size):
        X = []
        y = []
        
        for i in range(len(data) - window_size - 1):
            # Window includes all features
            X.append(data[i:(i + window_size), :])
            # Target is next day's closing price
            y.append(close_data[i + window_size])  # Closing price is first column
            
        return np.array(X), np.array(y).reshape(-1, 1)
    
    def build_lstm_model(self,X_train):
        model = Sequential([
            LSTM(units=128, 
            return_sequences=True, 
            input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        
        LSTM(units=64),
        Dropout(0.2),
        
        Dense(32, activation='relu'),
        Dense(1)
        ])
        optimizer = Adam(learning_rate=0.01)
        model.compile(optimizer=optimizer, loss='mean_squared_error')
        return model
    
    def build_mlp_model(self):
        model_mlp = MLPRegressor(hidden_layer_sizes=(128, 64), max_iter=100, verbose=True)
        return model_mlp
    
    def get_current_price(self):
        stock_data = yf.download(self.stock_name, period='2y',interval='1h')
        return stock_data['Close'].to_numpy()[-1]
    
    def train_lstm_univariate(self):
        scaler = MinMaxScaler()
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        print(df_windowed.shape,df_target.shape,df_windowed_reshaped.shape)
        x_scaler = scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        train_size = int(self.split_ratio*len(df_windowed))
        X_train = df_windowed[:train_size]
        y_train = df_target[:train_size]
        X_test = df_windowed[train_size:]
        y_test = df_target[train_size:]
        model = self.build_lstm_model(X_train)
        model.fit(X_train, y_train, epochs=100,batch_size=32,    
            validation_split=0.1,
            verbose=1)

        model.save(f'models/{self.stock_name}/LSTM_univariate.h5')
     
    def train_lstm_multivariante(self):
        scaler = MinMaxScaler()
        data = self.data[['Close', 'High', 'Low', 'Volume']]
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_multivariate(data.to_numpy(),data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        train_size = int(self.split_ratio*len(df_windowed))
        X_train = df_windowed[:train_size]
        y_train = df_target[:train_size]
        X_test = df_windowed[train_size:]
        y_test = df_target[train_size:]
        model = self.build_lstm_model(X_train)
        model.fit(X_train, y_train, epochs=100,batch_size=32,    
            validation_split=0.1,
            verbose=1)

        model.save(f'models/{self.stock_name}/LSTM_multivariate.h5')   
        
    def train_mlp_univariante(self):
        scaler = MinMaxScaler()
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        print(df_windowed.shape,df_target.shape,df_windowed_reshaped.shape)
        x_scaler = scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        train_size = int(self.split_ratio*len(df_windowed))
        X_train = df_windowed[:train_size]
        y_train = df_target[:train_size]
        X_test = df_windowed[train_size:]
        y_test = df_target[train_size:]
        model = self.build_mlp_model()
        model.fit(X_train.reshape(X_train.shape[0],-1), y_train)
        output_dir = f'models/{self.stock_name}'
        os.makedirs(output_dir, exist_ok=True)
        with open(f'models/{self.stock_name}/MLP_univaraite.pkl', 'wb') as file:
            pickle.dump(model, file)
    
    def model_expiry(self):
        EXPIRY_CST = 2*60
        models = ["MLP_univaraite","LSTM_univariate","LSTM_multivariate"]
        # check if json file with name models_manager exist in models folder 
        if not os.path.exists(f'models/{self.stock_name}/models_manager.json'):
            return True
        with open(f'models/{self.stock_name}/models_manager.json', 'r') as file:
            data = json.load(file)
            for model in models:
                if model not in data:
                    return True
                else:
                    creation_date = data[model]['creation_date']
                    creation_date = dt.datetime.fromtimestamp(creation_date)
                    now = dt.datetime.now()
                    diff = (now - creation_date).total_seconds()
                    print(diff)
                    return diff > EXPIRY_CST
                
    
    def train(self, force: bool = False):
        
        # TODO : detect the expiry of the model using the json file, json create name path of the model
        # diff( now - created ) > cst train else skip
        # if force is True, train the model
        if force or self.model_expiry():
            print("OK")
            self.train_lstm_univariate()
            self.train_lstm_multivariante()
            self.train_mlp_univariante()
            data = {
                    "MLP_univaraite":{
                        'path':f'models/{self.stock_name}/MLP_univaraite.pkl',
                        "creation_date":dt.datetime.now().timestamp()
                    },
                    "LSTM_univariate":{
                        'path':f'models/{self.stock_name}/LSTM_univariate.h5',
                        "creation_date":dt.datetime.now().timestamp()
                    },
                    "LSTM_multivariate":{
                        'path':f'models/{self.stock_name}/LSTM_multivariate.h5',
                        "creation_date":dt.datetime.now().timestamp()
                    }
                }
            with open(f'models/{self.stock_name}/models_manager.json', 'w') as file:
                json.dump(data, file)
            
    
    def forecast_lstm_univariante(self,n_hours:int=7):
        model_dir = f'models/{self.stock_name}/LSTM_univariate.h5'
        model = tf.keras.models.load_model(model_dir)
        ## forecast for n hours
        print(model_dir)
        forecast = []
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        for i in range(n_hours):
            X = X.reshape(1, X.shape[0], X.shape[1])
            y_pred = model.predict(X)
            forecast.append(y_pred)
            print(forecast)
            X = np.concatenate((X[0][1:], y_pred))
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    def forecast_lstm_multivariante(self,n_hours:int=7):
        model_dir = f'models/{self.stock_name}/LSTM_multivariate.h5'
        model = tf.keras.models.load_model(model_dir)
        ## forecast for n hours
        print(model_dir)
        forecast = []
        data = self.data[['Close', 'High', 'Low', 'Volume']]
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_multivariate(data.to_numpy(),data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        for i in range(n_hours):
            X = X.reshape(1, X.shape[0], X.shape[1])
            y_pred = model.predict(X)
            forecast.append(y_pred)
            y_pred_expanded = X[-1].copy()  # Use the last timestep as a template
            y_pred_expanded[0] = y_pred[0, 0]  # Update the 'Close' feature with the prediction
            X = np.concatenate((X[0][1:], y_pred_expanded), axis=0)
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    def forecast_mlp_univariate(self,n_hours):
        model_dir = f'models/{self.stock_name}/MLP_univaraite.pkl'
        with open(model_dir, 'rb') as file:
            model = pickle.load(file)
        data_close = self.data['Close']
        df_windowed,df_target = self.window_data_univariate(data_close.to_numpy().reshape(-1,1),self.window_size)
        df_windowed_reshaped = df_windowed.reshape(df_windowed.shape[0], -1)
        x_scaler = MinMaxScaler()
        x_scaler.fit(df_windowed_reshaped)
        df_windowed_reshaped = x_scaler.transform(df_windowed_reshaped)
        df_windowed = df_windowed_reshaped.reshape(df_windowed.shape[0], df_windowed.shape[1], df_windowed.shape[2])
        y_scaler = MinMaxScaler()
        y_scaler.fit(df_target)
        df_target = y_scaler.transform(df_target)
        X = df_windowed[-1]
        forecast = []
        for i in range(n_hours):
            y_pred = model.predict(X.reshape(1, -1)).reshape(-1, 1)
            forecast.append(y_pred)
            X = np.concatenate((X[1:], y_pred))
        return y_scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    
    def forecast_nhours(self,n_hours:int=7):
        return {
            'LSTM_univariate':self.forecast_lstm_univariante(n_hours),
            'LSTM_multivariate':self.forecast_lstm_multivariante(n_hours),
            'MLP_univariate':self.forecast_mlp_univariate(n_hours)
        }
    

        
        
            
            


In [3]:

    
class Predictor:
    def __init__(self,interval="1h",period="2y",stocks=['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA']):
        self.stocks = stocks
        self.predictors = {}
        for stock in stocks:
            self.predictors[stock] = StockPredictor(stock_name=stock,interval=interval,period=period)
        
    def train(self):
        for stock in self.predictors.values():
            stock.train()
    
    def forecast_nhours(self,n_hours:int=7):
        forecast = {}
        for stock in self.predictors.values():
            forecast[stock.stock_name] = stock.forecast_nhours(n_hours)
        return forecast
    
    def get_current_prices(self):
        prices = {}
        for stock in self.predictors.values():
            prices[stock.stock_name] = stock.get_current_price()
        return prices

In [4]:
test = Predictor()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
test.train()

1477.787923
OK
(3494, 6, 1)
(3494, 6, 1) (3494, 1) (3494, 6)
Epoch 1/100


c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


99/99 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0775 - val_loss: 8.1565e-04
Epoch 2/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 8.5873e-04 - val_loss: 8.6560e-04
Epoch 3/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.5250e-04 - val_loss: 0.0014
Epoch 4/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.7881e-04 - val_loss: 2.0980e-04
Epoch 5/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.7459e-04 - val_loss: 0.0011
Epoch 6/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.0599e-04 - val_loss: 3.2763e-04
Epoch 7/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.1240e-04 - val_loss: 6.5142e-04
Epoch 8/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.4032e-04 - val_loss: 1.9887e-04
Epoch 9/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.2299e-04 - val_loss: 8.1677e-04
Epoch 10/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.6132e-04 - val_loss: 2.6767e-04
Epoch 11/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.1833e-04 - val_loss: 7.0782e-0

Epoch 1/100


c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


99/99 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0177 - val_loss: 0.0132
Epoch 2/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.1292e-04 - val_loss: 0.0040
Epoch 3/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.8862e-04 - val_loss: 0.0059
Epoch 4/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.8559e-04 - val_loss: 0.0018
Epoch 5/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.6239e-04 - val_loss: 0.0022
Epoch 6/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.4295e-04 - val_loss: 0.0012
Epoch 7/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.1805e-04 - val_loss: 0.0020
Epoch 8/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.0356e-04 - val_loss: 0.0024
Epoch 9/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.4494e-04 - val_loss: 0.0035
Epoch 10/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.9787e-04 - val_loss: 0.0042
Epoch 11/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.7935e-04 - val_loss: 0.0025
Epoch 12/100
99/99 ━━━━━━━━━━━━━━━

(3494, 6, 1)
(3494, 6, 1) (3494, 1) (3494, 6)
Iteration 1, loss = 0.07529698


c:\Users\Dell Latitude 5420\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 2, loss = 0.00606858
Iteration 3, loss = 0.00106366
Iteration 4, loss = 0.00023880
Iteration 5, loss = 0.00017689
Iteration 6, loss = 0.00014833
Iteration 7, loss = 0.00013900
Iteration 8, loss = 0.00013295
Iteration 9, loss = 0.00012909
Iteration 10, loss = 0.00012673
Iteration 11, loss = 0.00012471
Iteration 12, loss = 0.00012322
Iteration 13, loss = 0.00012220
Iteration 14, loss = 0.00012080
Iteration 15, loss = 0.00011997
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [78]:
dt.datetime.now().timestamp()

1736102653.644085